In [1]:
import pandas as pd
import numpy as np

# Part I: Sync parsed userDf and parsed reviewDf
Sync userDfFile.csv with all reviewDfFile parsed from ParseUserJson and ParseReviewJson by counting the number of available reviews for each user, and collecting all available review_id for each user. 

In [48]:
#Load the user data frame file
userDf = pd.read_csv('userDfFile.csv',index_col=0)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


This is our first attempt. It is very slow due to the size of userDf.

In [ ]:
# '''Count the number of available reviews in reviewDf for each user in userDf and add that to the existing newUserDf which contains the available_review_count so far'''
# def countAvailableReview(userDf,reviewDf,newUserDf):
#     #loop through each user in userDf and count the number of reviews available in reviewDf
#     for i in range(len(userDf)):
#         user=userDf.loc[i,'user_id']
#         available_review_count=len(reviewDf.loc[reviewDf['user_id']==user])
#         newUserDf.at[i,'available_review_count']=available_review_count
#     return newUserDf

countAvailableReview() above will take O(N^2). Time complexity should decrease if we sort both userDf and reviewDf first and then sync. Sorting would take O(Nlogn) and syncing/counting would take O(N), the overall runtime should be O(N).

In [49]:
sortedUserDf=userDf.sort_values('user_id')

Create the function that will return two lists. The index (position-based) of each list matches that of sortedUserDf.

In [ ]:
def fastCountAvailReview(sortedUserDf,sortedReviewDf):
    '''For each user in sortedUserDf, count the number of reviews available in sortedReviewDf and put the review_id in a set. Return a list of available review counts, and another list of review_id s for each user_id.'''
    j=0 #iterator in sortedReviewDf
    jmax=len(sortedReviewDf)
    availReviewCol=[]
    reviewIdCol=[]
    for i in range(len(sortedUserDf)): #i <-- iterator in sortedUserDf
        #user_id we are currently syncing
        user=sortedUserDf.iloc[i]['user_id']
        #loop until we find a user_id >= user
        while (j<jmax and sortedReviewDf.iloc[j]['user_id']<user):
            j+=1
        reviewIdSet=set()
        #Count the number of available reviews in sortedReviewDf
        while (j<jmax and sortedReviewDf.iloc[j]['user_id']==user):
            reviewIdSet.add(sortedReviewDf.iloc[j]['review_id'])
            j+=1
        availReviewCol.append(len(reviewIdSet))
        reviewIdCol.append(reviewIdSet)
    return availReviewCol,reviewIdCol

## Call fastCountAvailReview to sync all 4 reviewDfFile*.csv
There are 4 reviewDfFile because the original review dataset is too large to put in one data frame.
### Sync reviewDf1.csv

In [ ]:
#Load the review data frame file
reviewDf=pd.read_csv('reviewDfFile1.csv',index_col=0)

In [ ]:
sortedReviewDf=reviewDf.sort_values('user_id')

In [ ]:
sortedReviewDf.tail()

There are rows that contain NaN at the last two rows, so we should drop them.

In [ ]:
#Drop rows that contain NaN
sortedReviewDf=sortedReviewDf.drop(sortedReviewDf.index[-2:])

In [ ]:
availReviewCol,reviewIdCol=fastCountAvailReview(sortedUserDf,sortedReviewDf)

In [ ]:
#Put the result in a data frame
availReviewDf1=pd.DataFrame(data={"AvailReviewCount":availReviewCol,"Reviews":reviewIdCol},index=sortedUserDf['user_id'])

### Sync ReviewDfFile2.csv

In [ ]:
#Load the review data frame file
reviewDf=pd.read_csv('reviewDfFile2.csv',index_col=0)

In [ ]:
sortedReviewDf=reviewDf.sort_values('user_id')

In [ ]:
sortedReviewDf.tail()

In [ ]:
#sortedReviewDf does not contain rows with NaN values, so proceed
availReviewCol,reviewIdCol=fastCountAvailReview(sortedUserDf,sortedReviewDf)

In [ ]:
#Put the results lists in a data frame and export it into a csv file
availReviewDf2=pd.DataFrame(data={"AvailReviewCount":availReviewCol,"Reviews":reviewIdCol},index=sortedUserDf['user_id'])

### Sync ReviewDfFile3.csv

In [ ]:
#Load the review data frame file
reviewDf=pd.read_csv('reviewDfFile3.csv',index_col=0)

In [ ]:
sortedReviewDf=reviewDf.sort_values('user_id')

In [ ]:
#sortedReviewDf does not contain rows with NaN values, so proceed
availReviewCol,reviewIdCol=fastCountAvailReview(sortedUserDf,sortedReviewDf)

In [ ]:
#Put the results lists in a data frame and export it into a csv file
availReviewDf3=pd.DataFrame(data={"AvailReviewCount":availReviewCol,"Reviews":reviewIdCol},index=sortedUserDf['user_id'])

### Sync ReviewDfFile4.csv

In [ ]:
#Load the review data frame file
reviewDf=pd.read_csv('reviewDfFile4.csv',index_col=0)

In [ ]:
sortedReviewDf=reviewDf.sort_values('user_id')

In [ ]:
sortedReviewDf.tail()

In [ ]:
#Drop rows that contain NaN
sortedReviewDf=sortedReviewDf.drop(sortedReviewDf.index[-2:])

In [ ]:
availReviewCol,reviewIdCol=fastCountAvailReview(sortedUserDf,sortedReviewDf)

In [ ]:
#Put the results lists in a data frame
availReviewDf4=pd.DataFrame(data={"AvailReviewCount":availReviewCol,"Reviews":reviewIdCol},index=sortedUserDf['user_id'])

# Now merge all availReveiewDf
The function belows take in a list of all availReviewDf, and the length of userDf as input. Length of each availReviewDf should equal len_user.

In [57]:
def mergeAvailReviewDf(Ldf,len_user):
    '''Merge data frames that contain available review count and available review_id. '''
    availReviewCountCol=[]
    reviewsCol=[]
    for i in range(len_user): #row number of the user (position-based)
        reviewset=Ldf[0].iloc[i]['Reviews']
        for j in range(1,len(Ldf)): #Ldf[j] <- dataframe syncing right now
            reviewset=reviewset.union(Ldf[j].iloc[i]['Reviews'])
        availReviewCountCol.append(len(reviewset))
        reviewsCol.append(reviewset)
    return availReviewCountCol, reviewsCol

In [ ]:
availReviewCountCol, reviewsCol=uniqueReviewCount([availReviewDf1,availReviewDf2,availReveiwDf3,availReviewDf4],1183362)

In [59]:
availReviewTotalDf=pd.DataFrame(data={"AvailReviewCount":availReviewCountCol,"Reviews":reviewsCol},index=sortedUserDf['user_id'])

In [60]:
availReviewTotalDf.head(reviewset=reviewset.union(eval(Ldf[j].iloc[i]['Reviews'])))

AvailReviewCount  \
user_id                                    
---1lKK3aKOuomHnwAkAow               117   
---94vtJ_5o_nikEs6hUjg                 1   
---PLwSf5gKdIoVnyRHgBA                 2   
---cu1hq55BP9DWVXXKHZg                 3   
---fhiwiwBYrvqhpXgcWDQ                 1   

                                                                  Reviews  
user_id                                                                    
---1lKK3aKOuomHnwAkAow  {_aSUxNefC6eiakBZlmfrkQ, X7-Kqkzg2J4rpKJGXvXfU...  
---94vtJ_5o_nikEs6hUjg                           {bImuLGmVMhUTjFinMjM46Q}  
---PLwSf5gKdIoVnyRHgBA   {5Y_hKqpo4wb1pPtFgtCqrA, QXzgdKOdmrccxZ60Er7gig}  
---cu1hq55BP9DWVXXKHZg  {C64IMmOZmJWiNdFe3ELklQ, VaekJKjFRtjozEcf9WTvC...  
---fhiwiwBYrvqhpXgcWDQ                           {ek_TJCDJ3vjKqfcnKEgtHg}

In [61]:
availReviewTotalDf.to_csv('availReviewDf.csv')

Check the percentage of available reviews for all users in the user dataset, drop rows with <= 80% of reviews available

In [66]:
import copy
newSortedUserDf=copy.deepcopy(sortedUserDf)

In [ ]:
availPct=[] #stores percentage of available reviews for each user
dropidx=[]
for i in range(len(newSortedUserDf)):
    actual=newSortedUserDf.iloc[i]['review_count']
    if actual==0:
        pct=0
    else:
        pct=availReviewTotalDf.iloc[i]['AvailReviewCount']/actual
    availPct.append(pct)
    if (pct<=0.8):
        dropidx.append(i)

In [93]:
newSortedUserDf['AvailPct']=availPct

In [94]:
newSortedUserDf=newSortedUserDf.drop(newSortedUserDf.index[dropidx])

In [95]:
availReviewTotalDf=availReviewTotalDf.drop(availReviewTotalDf.index[dropidx])

Attach the column that contains the percentage of available reviews, and the colomn of all available review_id to the sorted user data frame.

In [120]:
newSortedUserDf['AvailReviewCount']=availReviewTotalDf['AvailReviewCount'].values

In [123]:
newSortedUserDf['Reviews']=availReviewTotalDf['Reviews'].values

In [124]:
newSortedUserDf.tail()

average_stars  review_count                 user_id yelping_since  \
1093646           4.25             1  zzvm_VD3HjU6RMx8zq7g1g    2015-06-12   
396025            5.00             1  zzvqVZTYs5VKxPc-IkRQ4A    2017-03-15   
571955            2.67             3  zzwTJAgONYCTeJia4te47g    2016-04-24   
1178718           5.00             1  zzy9H_45XLydmYZ2P_Ve_g    2014-08-27   
810511            3.00             2  zzyMMeUZzKAy7KQhM7lU2w    2013-12-06   

         AvailPct  AvailReviewCount  \
1093646       1.0                 1   
396025        1.0                 1   
571955        1.0                 3   
1178718       1.0                 1   
810511        1.0                 2   

                                                   Reviews  
1093646                           {nd8ZaTy3wcrfLvuKTN1jYw}  
396025                            {simX0wWLHTOpj_IyQBKeAQ}  
571955   {Zw1hLwD6gYQ4Z15Fo8n6Tg, J6sDT53hNjtvUZvk34Jrh...  
1178718                           {dVC4-hUXF4qbXAi9sG3lsg}  
810511    {KIwAs2hgBdmSrUM5M7o_OA, Qh0nbp8yyRLFkqlBiVIxqw}

In [130]:
newSortedUserDf.describe()

average_stars   review_count       AvailPct  AvailReviewCount
count  388904.000000  388904.000000  388904.000000     388904.000000
mean        3.623829       4.380855       0.985101          4.066556
std         1.477098      13.790242       0.045614         12.017385
min         1.000000       1.000000       0.800866          1.000000
25%         2.630000       1.000000       1.000000          1.000000
50%         4.000000       1.000000       1.000000          1.000000
75%         5.000000       3.000000       1.000000          3.000000
max         5.000000    1178.000000       8.000000        948.000000

In [131]:
newSortedUserDf.dtypes

average_stars       float64
review_count          int64
user_id              object
yelping_since        object
AvailPct            float64
AvailReviewCount      int64
Reviews              object
dtype: object

In [125]:
len(newSortedUserDf)

388904

Export the data frame.

In [126]:
newSortedUserDf.to_pickle('UserWithReviewDf.pkl')

Somehow some users have AvailReviewCount greater than review_count. This doesn't make sense, so we drop these users.

In [2]:
userDf=pd.read_pickle('UserWithReviewDf.pkl')

In [3]:
wrongreview=userDf.loc[userDf['review_count']<userDf['AvailReviewCount']]

In [4]:
wrongreview

average_stars  review_count                 user_id yelping_since  \
651597            2.50             2  -CgbL7Rygee52C3ylCV3iQ    2017-01-06   
672767            3.80             1  09z9No52Jiyf1y4sO31GQw    2012-05-02   
360584            2.06            19  0LQZIzeAsJgLdrRp4wd3zA    2014-07-02   
338210            4.25             6  0imvl3fmbRdpF_CD36D1JA    2013-01-01   
162375            2.40             6  2y9cPgLYd4Okx6Zk1EBvsA    2012-07-31   
617439            3.00             1  3Hv7dnRcwC4IBEvUyIj6UQ    2012-04-17   
526622            3.00             1  4guECw0hnVUijdv8RnSzUg    2014-11-16   
664959            1.75             6  4uOzvmj_TQcWwCZfdTrimA    2012-02-08   
296721            3.60             3  6cba0fI0-Hog52YFLAYN3A    2015-09-08   
267796            2.00             3  7RL3qW5IqoD4Mu3X1bkjjA    2016-08-17   
191026            5.00             3  87v53p49Kh_m7caV8KhAbQ    2014-08-08   
199891            3.00             5  C_LxqoDy2nwwTZX7XT5LHg    2014-05-09   
459704            2.82             3  CqO8EBO8K7wV6cyAvDKN6A    2016-09-26   
644384            2.33             1  E019O2XqmdwEma5pPZY4wQ    2016-03-21   
265598            1.58            12  ERolBvFzLKfqq8DouenfOA    2013-06-01   
33209             3.79            73  EdDmKiz6V89g6Dws9I4AKA    2016-01-27   
300370            3.17             5  F1keniVFFw88gOiEuuSN9A    2015-04-15   
5366              3.84             1  GXnPGaeM334u_e_JCoB1kA    2009-07-26   
56290             3.38            14  I6eqcO4AA6jiCoak1iX-Mg    2012-11-15   
55476             2.17            27  IPor3JMA74Ig1LivOt0d3Q    2008-11-12   
17689             5.00             3  M47skZU1iimnrpz5Fb2HuA    2014-10-11   
266726            2.63             5  MbChyRFmIjDw6N09HCJORw    2016-11-23   
693101            5.00             2  MqjTaPHdrotau9Pfmv1Tcg    2013-05-16   
617568            3.00             1  N3_acAlXyiaOl8MSu2Knww    2017-02-05   
368515            4.86             1  Pfkoyhdup4UeFOslJDLS1Q    2015-02-21   
538607            5.00             2  QPFyfh2vkdWs2aNrFqb9mA    2017-01-17   
131668            4.17             3  Qg42D4N2l-tUBJ1eXKYHaw    2014-06-22   
142661            4.14             9  RGKIVa5bGSqX5vMnpBa9Fw    2014-09-01   
490818            4.25             2  S0hQ2Xbfz1R9c6a69arYWQ    2016-10-23   
1009629           4.00             2  UnP006Mbi-AqSTn23SzyWw    2013-07-18   
...                ...           ...                     ...           ...   
632106            4.67             1  Y5iBlmisRqCGgVhDMXFobg    2016-10-26   
622056            3.67             2  ZegUGw7AUB0fjJDEyI82kg    2016-07-16   
247760            1.00             1  _apv24N3nLwIdPrHxmjNmQ    2017-01-22   
383555            3.00             5  _wYJcDJ8YWCqFdU3C8wHXQ    2014-11-17   
121759            3.00             2  bgmffI7VNYbia4yOZ4urqg    2012-10-25   
22103             3.24            11  ddPFBOsvFjaOVRj8sTZQmA    2014-06-26   
177654            3.67             6  elf4PucDzGfsKk4psaf2Qw    2016-07-18   
235694            3.44            10  fL5CQ4s4ANUvwsU0h1y5Ag    2016-05-26   
58404             3.25            15  hOPE0YPENSZhsC-nlpxIRQ    2015-07-23   
539190            3.67             8  hbQobvcC07W0zQA_93sxKA    2013-03-01   
598738            4.00             5  jDoR1VKY9mnWur0YVdF2HQ    2013-10-21   
724756            3.67             2  kQG_KRMfDX5GLCMAzu5MLQ    2013-08-23   
717658            2.56             1  limKeYzmVfivoDpZNs2qzA    2014-03-20   
550143            3.67             2  oP_guLe__P5BKsTMcV_DDA    2016-11-10   
276321            3.33             1  oS87-mKITgMupgV12hmQxw    2015-02-14   
350345            2.80             3  pQSpJukPH5QmvebFHgJc7A    2016-09-22   
90758             3.86             6  qNeb0IsEKPR6K9IgtOcVcg    2014-08-27   
539814            2.60             2  qvyN8O2-U-hitMZypmKF1w    2014-11-04   
246059            3.00             2  sLbjJ2OTQOLSnMJLABFGPA    2016-11-14   
133710            3.92         

In [5]:
userDf=userDf.drop(wrongreview.index)

In [6]:
userDf.describe()

average_stars   review_count       AvailPct  AvailReviewCount
count  388841.000000  388841.000000  388841.000000     388841.000000
mean        3.623855       4.380572       0.984999          4.065978
std         1.477172      13.790725       0.043293         12.017532
min         1.000000       1.000000       0.800866          1.000000
25%         2.630000       1.000000       1.000000          1.000000
50%         4.000000       1.000000       1.000000          1.000000
75%         5.000000       3.000000       1.000000          3.000000
max         5.000000    1178.000000       1.000000        948.000000

# Part II: Pick a subset of all active users and inactive users
It's impossible to analyze all users (because there are so many), so we are more interested in only analyzing active users and inactive users. We pick active users as those with *review_count>=90*, and inactive users as those with *review_count<=1*.

In [7]:
active=userDf.loc[userDf['review_count']>=90]
active.describe()

average_stars  review_count     AvailPct  AvailReviewCount
count    1478.000000   1478.000000  1478.000000       1478.000000
mean        3.718018    163.820704     0.873158        142.470907
std         0.380821    103.633382     0.048381         88.343739
min         1.870000     90.000000     0.800866         73.000000
25%         3.490000    104.000000     0.833333         92.000000
50%         3.730000    129.000000     0.866050        113.000000
75%         3.960000    180.000000     0.908182        156.000000
max         4.810000   1178.000000     1.000000        948.000000

In [8]:
active['AvailReviewCount'].sum()

210572

Because we are interested in analyzing the correlation between user activity and review texts, 200k is already a significant number of reviews to analyze.

In [9]:
inactive=userDf.loc[userDf['review_count']<=1]
inactive.describe()

average_stars  review_count  AvailPct  AvailReviewCount
count  214054.000000      214054.0  214054.0          214054.0
mean        3.586616           1.0       1.0               1.0
std         1.746058           0.0       0.0               0.0
min         1.000000           1.0       1.0               1.0
25%         1.000000           1.0       1.0               1.0
50%         5.000000           1.0       1.0               1.0
75%         5.000000           1.0       1.0               1.0
max         5.000000           1.0       1.0               1.0

In [10]:
inactive['AvailReviewCount'].sum()

214054

Export the data frames of active users and inactive users

In [ ]:
inactive.to_pickle('inactiveUserDf.pkl')
active.to_pickle('activeUserDf.pkl')

### Get all reviews for active users and inactive users
Create a function to get all review_id for active users and inactive users, so that we can fetch the corresponding review texts later.

In [ ]:
def getReviews(df):
    '''Return all review_id for df in a list, assuming there are no duplicates in the review_id'''
    reviews=[]
    for reviewset in df['Reviews']:
        for review in reviewset:
            reviews.append(review)
    return reviews

In [ ]:
inactiveReviews=getReviews(inactive)

In [ ]:
activeReviews=getReviews(active)

In [ ]:
def groupByActivity(csvfile,inactiveReviews,activeReviews):
    '''Read in a csv file and return a data frame that contains all reviews that belong to inactive users, and another data frame that contains all reviews that belong to active users'''
    reviewDf=pd.read_csv(csvfile,index_col=0)
    inactiveDf=reviewDf.loc[reviewDf['review_id'].isin(inactiveReviews)]
    activeDf=reviewDf.loc[reviewDf['review_id'].isin(activeReviews)]
    return inactiveDf,activeDf

Go through all parsed review data frames and fetch review data for inactive users and active users.

In [ ]:
inactiveDf1,activeDf1=groupByActivity('reviewDfFile1.csv',inactiveReviews,activeReviews)

In [ ]:
inactiveDf2,activeDf2=groupByActivity('reviewDfFile2.csv',inactiveReviews,activeReviews)

In [ ]:
inactiveDf3,activeDf3=groupByActivity('reviewDfFile3.csv',inactiveReviews,activeReviews)

In [ ]:
inactiveDf4,activeDf4=groupByActivity('reviewDfFile4.csv',inactiveReviews,activeReviews)

Merge all 4 data frames into 1 data frame for active users and inactive users respectively.

In [ ]:
inactiveDf=pd.concat([inactiveDf1,inactiveDf2,inactiveDf3,inactiveDf4])

In [ ]:
activeDf=pd.concat([activeDf1,activeDf2,activeDf3,activeDf4])

Convert stars from float to int and export inactiveDf and activeDf.

In [ ]:
inactiveDf['stars']=inactiveDf['stars'].astype(int)
activeDf['stars']=activeDf['stars'].astype(int)

In [ ]:
activeDf.to_pickle('activeReviewDf.pkl')

In [ ]:
inactiveDf.to_pickle('inactiveReivewDf.pkl')

# Part III: split the data into a train set and a test set

In [ ]:
from sklearn.model_selection import train_test_split
train1, test1 = train_test_split(activeDf, test_size=0.2)

In [ ]:
train2, test2 = train_test_split(inactiveDf, test_size=0.2)

In [ ]:
train=pd.concat([train1,train2])
test=pd.concat([test1,test2])

In [ ]:
train.to_pickle('train.pkl')

In [ ]:
test.to_pickle('test.pkl')